In [1]:
import os

In [2]:
os.chdir('../')

In [3]:
%pwd

'/Users/mansipandya/Desktop/Lung-Cancer-Prediction'

In [4]:
import mlflow
import mlflow.sklearn

In [5]:
import dagshub

dagshub.init(repo_owner='mansipandyaa29',
             repo_name='Lung-Cancer-Prediction',
             mlflow=True)

Initialized MLflow to track repo "mansipandyaa29/Lung-Cancer-Prediction"

Repository mansipandyaa29/Lung-Cancer-Prediction initialized!

In [6]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class ModelEvaluationConfig:
    root_dir: Path
    test_data_path: Path
    model_path: Path
    all_params: dict
    metric_file_name: Path
    target_column: str
    mlflow_uri: str

In [7]:
from LungCancerPrediction.constants import *
from LungCancerPrediction.utils.common import read_yaml, create_directories, save_json

In [8]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])

    
    def get_model_evaluation_config(self) -> ModelEvaluationConfig:
        config = self.config.model_evaluation
        params = self.params.RandomForestClassifier
        schema =  self.schema.TARGET_COLUMN

        create_directories([config.root_dir])

        model_evaluation_config = ModelEvaluationConfig(
            root_dir=config.root_dir,
            test_data_path=config.test_data_path,
            model_path = config.model_path,
            all_params=params,
            metric_file_name = config.metric_file_name,
            target_column = schema.name,
            mlflow_uri="https://dagshub.com/mansipandyaa29/Lung-Cancer-Prediction.mlflow",
           
        )

        return model_evaluation_config

In [9]:
import os
import pandas as pd
from sklearn.metrics import precision_score, recall_score, accuracy_score, r2_score, f1_score
from urllib.parse import urlparse
import mlflow
import mlflow.sklearn
import numpy as np
import joblib

In [10]:
class ModelEvaluation:
    def __init__(self, config: ModelEvaluationConfig):
        self.config = config

    
    def eval_metrics(self,actual, pred):
        precision = precision_score(actual, pred, average = 'micro')
        recall = recall_score(actual, pred, average = 'micro')
        accuracy = accuracy_score(actual, pred)
        f1 = f1_score(actual, pred, average = 'micro')
        r2 = r2_score(actual, pred)
        return precision, recall, accuracy, f1, r2

    def log_into_mlflow(self):

        test_data = pd.read_csv(self.config.test_data_path)
        model = joblib.load(self.config.model_path)

        test_x = test_data.drop([self.config.target_column], axis=1)
        test_y = test_data[[self.config.target_column]]


        mlflow.set_registry_uri(self.config.mlflow_uri)
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme


        with mlflow.start_run():

            predicted_qualities = model.predict(test_x)

            (precision, recall, accuracy, f1, r2) = self.eval_metrics(test_y, predicted_qualities)
            
            # Saving metrics as local
            scores = {"precision": precision, "recall": recall, "accuracy": accuracy, "f1": f1, "r2": r2}
            save_json(path=Path(self.config.metric_file_name), data=scores)

            mlflow.log_params(self.config.all_params)

            mlflow.log_metric("precision", precision)
            mlflow.log_metric("recall", recall)
            mlflow.log_metric("accuracy", accuracy)
            mlflow.log_metric("f1", f1)
            mlflow.log_metric("r2", r2)


            # Model registry does not work with file store
            if tracking_url_type_store != "file":
                mlflow.sklearn.log_model(model, "model", registered_model_name="RandomForestClassifier")
            else:
                mlflow.sklearn.log_model(model, "model")

In [11]:
try:
    config = ConfigurationManager()
    model_evaluation_config = config.get_model_evaluation_config()
    model_evaluation_config = ModelEvaluation(config=model_evaluation_config)
    model_evaluation_config.log_into_mlflow()
except Exception as e:
    raise e

[2024-06-04 11:41:31,165: INFO: common: yaml file: config/config.yaml loaded successfully]
[2024-06-04 11:41:31,166: INFO: common: yaml file: params.yaml loaded successfully]
[2024-06-04 11:41:31,169: INFO: common: yaml file: schema.yaml loaded successfully]
[2024-06-04 11:41:31,169: INFO: common: created directory at: artifacts]
[2024-06-04 11:41:31,170: INFO: common: created directory at: artifacts/model_evaluation]
[2024-06-04 11:41:31,620: INFO: common: json file saved at: artifacts/model_evaluation/metrics.json]


/Users/mansipandya/anaconda3/envs/lungcancerds/lib/python3.10/site-packages/_distutils_hack/__init__.py:26: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")
Registered model 'RandomForestClassifier' already exists. Creating a new version of this model...
2024/06/04 11:41:38 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation.                     Model name: RandomForestClassifier, version 6
Created version '6' of model 'RandomForestClassifier'.
